In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
urlist=['https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/page-blocks-1-3_vs_4.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/ecoli4.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/poker-8_vs_6.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/winequality-red-8_vs_6.dat','https://raw.githubusercontent.com/sydney-machine-learning/GANclassimbalanced/main/DATASETS/yeast-2_vs_4.dat']
option=int(input("Enter 0 for page-blocks\nEnter 1 for ecoli\nEnter 2 for poker\nEnter 3 for winequality\nEnter 4 for yeast\n"))


Enter 0 for page-blocks
Enter 1 for ecoli
Enter 2 for poker
Enter 3 for winequality
Enter 4 for yeast
 2


In [3]:
import pandas as pd
url=urlist[option]
data = pd.read_csv(url, sep=",", header='infer' )
data

,4,5,1,9,3,5.1,4.1,9.1,2,9.2,negative
0,4,11,4,5,2,11,3,11,1,5,negative
1,3,1,4,1,3,4,1,4,2,1,negative
2,3,5,2,5,3,1,2,1,4,1,negative
3,4,13,2,13,4,7,3,7,1,13,negative
4,2,10,3,12,3,10,1,12,1,10,negative
...,...,...,...,...,...,...,...,...,...,...,...
1471,4,10,2,11,3,11,3,10,1,10,negative
1472,1,4,3,3,4,4,1,3,2,4,negative
1473,1,10,3,3,3,10,2,10,4,3,negative
1474,1,6,2,6,4,12,3,6,1,12,negative


In [4]:
t=()
t=data.shape
X = data.values[:,0:(t[1]-1)].astype(float)
Y = data.values[:,(t[1]-1)]
print(X)

[[ 4. 11.  4. ... 11.  1.  5.]
 [ 3.  1.  4. ...  4.  2.  1.]
 [ 3.  5.  2. ...  1.  4.  1.]
 ...
 [ 1. 10.  3. ... 10.  4.  3.]
 [ 1.  6.  2. ...  6.  1. 12.]
 [ 3.  4.  1. ...  6.  3.  6.]]


In [5]:
if option==0 or option==3 or option==2 :
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    print(X)

[[1.         0.83333333 1.         ... 0.83333333 0.         0.33333333]
 [0.66666667 0.         1.         ... 0.25       0.33333333 0.        ]
 [0.66666667 0.33333333 0.33333333 ... 0.         1.         0.        ]
 ...
 [0.         0.75       0.66666667 ... 0.75       1.         0.16666667]
 [0.         0.41666667 0.33333333 ... 0.41666667 0.         0.91666667]
 [0.66666667 0.25       0.         ... 0.41666667 0.66666667 0.41666667]]


In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)
ysi=pd.Series(encoded_Y) 
ysi.value_counts()

0    1459
1      17
dtype: int64

In [7]:
if option==1:
    yk=[]
    for i in encoded_Y:
        if i==2:
            yk.append(1)
        else:
            yk.append(0)
    encoded_Y = np.asarray(yk, dtype=np.float32)
    encoded_Y.shape

In [8]:
if option==0:
    rs=1
    ep=10
    ne=2000
if option==1:
    rs=0
    ep=10
    ne=700
if option==2:
    rs=1
    ep=30
    ne=1500
if option==3:
    rs=4
    ep=30
    ne=1500    
if option==4:
    rs=1
    ep=10
    ne=1000    
    
    

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,encoded_Y, test_size=0.2, random_state=rs)

In [10]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))
from imblearn.over_sampling import SMOTE
X_train_res,y_train_res = SMOTE().fit_resample(X_train,y_train)

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))

Before OverSampling, counts of label '1': 14
Before OverSampling, counts of label '0': 1166 

After OverSampling, the shape of train_X: (2332, 10)
After OverSampling, the shape of train_y: (2332,) 

After OverSampling, counts of label '1': 1166
After OverSampling, counts of label '0': 1166


In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
from sklearn.metrics import f1_score
from statistics import stdev

2.4.1


In [12]:
def callf1(xx,yy,xt,yt):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam',
                  loss='mean_absolute_error',
                  metrics=['accuracy'])
    model.fit(xx, yy , epochs=ep)
    ls=[]
    test_loss, test_acc = model.evaluate(xt,  yt, verbose=2)
    print('\nTest accuracy:', test_acc)
    tr_loss, tr_acc = model.evaluate(xx,  yy, verbose=2)
    ls.append(test_acc)
    ls.append(tr_acc)
    ypr=model.predict(xt)
    ypr=(ypr>0.5)*1
    ypre= np.ravel(ypr)
    ls.append(f1_score(yt, ypre))
    return ls

In [13]:
sta=[]
ste=[]
sf =[]
for i in range(30):
    r=callf1(X_train, y_train.ravel(),X_test,y_test.ravel())
    ste.append(r[0])
    sta.append(r[1])
    sf.append(r[2])

Epoch 1/30
37/37 [==============================] - 1s 2ms/step - loss: 0.0448 - accuracy: 0.9870
Epoch 2/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0240 - accuracy: 0.9908
Epoch 3/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0189 - accuracy: 0.9916
Epoch 4/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0212 - accuracy: 0.9871
Epoch 5/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0183 - accuracy: 0.9905
Epoch 6/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0155 - accuracy: 0.9908
Epoch 7/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0183 - accuracy: 0.9881
Epoch 8/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0155 - accuracy: 0.9902
Epoch 9/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 0.9856
Epoch 10/30
37/37 [==============================] - 0s 2ms/step - loss: 0.0182 - accuracy: 0.9891
Epoch 11/30
37/37 [

In [14]:
sum(sf)/30

0.6300000000000002

In [15]:
sum(sta)/30

0.9902259747187296

In [16]:
sum(ste)/30

0.9949324190616607

In [17]:
max(sf)

0.8

In [18]:
max(sta)

0.9906779527664185

In [19]:
max(ste)

0.9966216087341309

In [20]:
stdev(sf)

0.2534588315887707

In [21]:
stdev(sta)

0.0008542502609854777

In [22]:
stdev(ste)

0.0023050230723720166

In [23]:
sta2=[]
ste2=[]
sf2 =[]
for i in range(30):
    r=callf1(X_train_res,y_train_res.ravel(),X_test,y_test.ravel())
    ste2.append(r[0])
    sta2.append(r[1])
    sf2.append(r[2])

Epoch 1/30
73/73 [==============================] - 1s 3ms/step - loss: 0.4065 - accuracy: 0.6671
Epoch 2/30
73/73 [==============================] - 0s 2ms/step - loss: 0.1440 - accuracy: 0.9894
Epoch 3/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0867 - accuracy: 1.0000
Epoch 4/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0794 - accuracy: 0.9988
Epoch 5/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0665 - accuracy: 1.0000
Epoch 6/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0613 - accuracy: 1.0000
Epoch 7/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0569 - accuracy: 1.0000
Epoch 8/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0490 - accuracy: 1.0000
Epoch 9/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0436 - accuracy: 1.0000
Epoch 10/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0517 - accuracy: 1.0000
Epoch 11/30
73/73 [

In [24]:
sum(sf2)/30

1.0

In [25]:
sum(sta2)/30

1.0

In [26]:
sum(ste2)/30

1.0

In [27]:
max(sf2)

1.0

In [28]:
max(sta2)

1.0

In [29]:
max(ste2)

1.0

In [30]:
stdev(sf2)

0.0

In [31]:
stdev(sta2)

0.0

In [32]:
stdev(ste2)

0.0

In [33]:
t2=X_train.shape

In [34]:
X_oversampled=X_train_res[(t2[0]):]

In [35]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [36]:
torch.cuda.empty_cache()

In [37]:
def get_generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True),
    )
class Generator(nn.Module):

    def __init__(self, z_dim=t2[1], im_dim=t2[1], hidden_dim=128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            get_generator_block(z_dim, hidden_dim),
            get_generator_block(hidden_dim, hidden_dim * 2),
            get_generator_block(hidden_dim * 2, hidden_dim * 4),
            get_generator_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        return self.gen(noise)
    
    # Needed for grading
    def get_gen(self):

        return self.gen
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(0.2, inplace=True)        
    )
class Discriminator(nn.Module):
    def __init__(self, im_dim=t2[1], hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):

        return self.disc(image)
    
    def get_disc(self):

        return self.dis
X_oversampled = torch.from_numpy(X_oversampled)
criterion = nn.BCEWithLogitsLoss()
n_epochs = ne
z_dim = t2[1]
batch_size = 128
lr = 0.00001
display_step = 1
device = 'cuda'
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr)
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr)
def get_disc_loss(gen, disc, criterion, real, device):

    fake = gen(X_oversampled.float().to(device))
    disc_fake_pred = disc(fake.detach())
    disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
    disc_real_pred = disc(real)
    disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
    disc_loss = (disc_fake_loss + disc_real_loss) / 2
    return disc_loss
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    fake_images = gen(X_oversampled.float().to(device))
    
    disc_fake_pred = disc(fake_images)
    gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
    return gen_loss
y_tr=y_train.ravel()
li=[]
for i in range(len(y_tr)):
    if int(y_tr[i])==1:
        li.append(X_train[i])
X_real=np.array(li)
t3=X_real.shape
li2=[1]*(t3[0])
y_real=np.array(li2)
y_real.shape
from torch.utils.data import TensorDataset, DataLoader
tensor_x = torch.Tensor(X_real) 
tensor_y = torch.Tensor(y_real)
my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
dataloader = DataLoader(
    my_dataset,
    batch_size=batch_size,
    shuffle=True)
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
test_generator = True 
gen_loss = False
error = False
for epoch in range(n_epochs):
  
    # Dataloader returns the batches
    for real, _ in tqdm(dataloader):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        real = real.view(cur_batch_size, -1).to(device)

        ### Update discriminator ###
        # Zero out the gradients before backpropagation
        disc_opt.zero_grad()

        # Calculate discriminator loss
        disc_loss = get_disc_loss(gen, disc, criterion, real, device)

        # Update gradients#
        disc_loss.backward(retain_graph=True)

        # Update optimizer
        disc_opt.step()

        # For testing purposes, to keep track of the generator weights
        if test_generator:
            old_generator_weights = gen.gen[0][0].weight.detach().clone()

        gen_opt.zero_grad()
        gen_loss = get_gen_loss(gen, disc, criterion, cur_batch_size, z_dim, device)
        gen_loss.backward()
        gen_opt.step()        
        #### END CODE HERE ####

        # For testing purposes, to check that your code changes the generator weights
        if test_generator:
            try:
                assert lr > 0.0000002 or (gen.gen[0][0].weight.grad.abs().max() < 0.0005 and epoch == 0)
                assert torch.any(gen.gen[0][0].weight.detach().clone() != old_generator_weights)
            except:
                error = True
                print("Runtime tests have failed")

        # Keep track of the average discriminator loss
        mean_discriminator_loss += disc_loss.item() / display_step

        # Keep track of the average generator loss
        mean_generator_loss += gen_loss.item() / display_step

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1, step 1: Generator loss: 1.3862016797065735, discriminator loss: 1.3823652267456055


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2, step 2: Generator loss: 0.6933953166007996, discriminator loss: 0.691048800945282


...

Epoch 1497, step 1497: Generator loss: 0.7010803818702698, discriminator loss: 0.6925631761550903


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1498, step 1498: Generator loss: 0.7012668251991272, discriminator loss: 0.69251948595047


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1499, step 1499: Generator loss: 0.7014461159706116, discriminator loss: 0.692473292350769


In [38]:
res=gen(X_oversampled.float().to(device))

In [39]:
res

tensor([[0.9735, 0.4850, 0.9531,  ..., 0.4806, 0.9567, 0.5539],
        [0.0047, 0.4775, 0.0056,  ..., 0.2809, 0.0053, 0.5690],
        [0.9885, 0.1235, 0.9829,  ..., 0.2435, 0.9742, 0.4228],
        ...,
        [0.9870, 0.5157, 0.9773,  ..., 0.5508, 0.9801, 0.6192],
        [0.0049, 0.5185, 0.0080,  ..., 0.5330, 0.0059, 0.7294],
        [0.9890, 0.4029, 0.9849,  ..., 0.4343, 0.9836, 0.6189]],
       device='cuda:0', grad_fn=<SigmoidBackward>)

In [40]:
X_oversampled

tensor([[0.2528, 0.5201, 0.2528,  ..., 0.4340, 0.2528, 0.6035],
        [1.0000, 0.3881, 1.0000,  ..., 0.5117, 1.0000, 0.6783],
        [0.3333, 0.3782, 0.3333,  ..., 0.3130, 0.3333, 0.5224],
        ...,
        [0.0000, 0.5417, 0.0000,  ..., 0.3333, 0.0000, 0.3335],
        [0.7623, 0.1783, 0.7623,  ..., 0.0478, 0.7623, 0.1550],
        [0.0000, 0.5234, 0.0000,  ..., 0.4504, 0.0000, 0.3670]],
       dtype=torch.float64)

In [41]:
fres=res.cpu().detach().numpy()
fres.shape

(1152, 10)

In [42]:
tu=X_train.shape

In [43]:
fin=np.concatenate((X_train_res[:(tu[0])], fres), axis=0)

In [44]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [45]:
y_train_res.shape

(2332,)

In [46]:
Xn,yn=shuffle_in_unison(fin, y_train_res)

In [47]:
ste3=[]
sta3=[]
sf3=[]
for i in range(30):
    r=callf1(Xn, yn.ravel(),X_test,  y_test.ravel())
    ste3.append(r[0])
    sta3.append(r[1])
    sf3.append(r[2])

Epoch 1/30
73/73 [==============================] - 0s 2ms/step - loss: 0.3525 - accuracy: 0.7199
Epoch 2/30
73/73 [==============================] - 0s 2ms/step - loss: 0.1404 - accuracy: 0.9807
Epoch 3/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0942 - accuracy: 0.9917
Epoch 4/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0700 - accuracy: 0.9952
Epoch 5/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0532 - accuracy: 1.0000
Epoch 6/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0445 - accuracy: 1.0000
Epoch 7/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0423 - accuracy: 1.0000
Epoch 8/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0347 - accuracy: 1.0000
Epoch 9/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0305 - accuracy: 1.0000
Epoch 10/30
73/73 [==============================] - 0s 2ms/step - loss: 0.0272 - accuracy: 1.0000
Epoch 11/30
73/73 [

In [58]:
sum(sf3)/30

1.0

In [59]:
sum(sta3)/30

1.0

In [60]:
sum(ste3)/30

1.0

In [61]:
max(sf3)

1.0

In [62]:
max(sta3)

1.0

In [63]:
max(ste3)

1.0

In [64]:
stdev(sf3)

0.0

In [65]:
stdev(sta3)

0.0

In [66]:
stdev(ste3)

0.0

In [67]:
save=1

In [68]:
progress=1

In [69]:
save=2